### Imports:

In [ ]:
import matplotlib
import pandas as pd
import numpy as np
import mne

### Loading the file:

In [ ]:
bdf_path = r"J:/Workgroups/FSW/VISUAL-DECISIONS/subjects_2025/103/103.bdf"

raw_eeg = mne.io.read_raw_bdf(bdf_path, preload=True)
raw_eeg.info

### Filtering channels out:

In [ ]:
biosemi_montage = mne.channels.make_standard_montage("biosemi32")
raw_eeg.set_montage(biosemi_montage, on_missing='ignore')
raw_eeg.set_channel_types(
    {'EXG1': 'eog', 'EXG2': 'eog', 'EXG3': 'eog', 'EXG4': 'eog', 'EXG5': 'eog', 'EXG6': 'eog'})
raw_eeg.drop_channels(['GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp'])  # Unused channels
raw_eeg.drop_channels(['EXG7', 'EXG8'])  # Unused EOG channels

In [ ]:
raw_eeg.plot_sensors(show_names=True)

In [ ]:
raw_eeg.set_eeg_reference(ref_channels=['EXG5', 'EXG6'])  # Take average of mastoids as reference

In [ ]:
mne.viz.plot_raw(raw_eeg)

## Filtering:

In [ ]:
raw_eeg = raw_eeg.filter(l_freq=0.1, h_freq=30)

In [ ]:
mne.viz.plot_raw(raw_eeg)

In [ ]:
%matplotlib inline
raw_eeg.compute_psd(picks="data", exclude="bads", fmin=0.1, fmax = 40, tmin=207, tmax=1370).plot(average=True, amplitude=False)

## ICA:

In [ ]:
from mne.preprocessing import ICA, create_eog_epochs


ica = ICA(n_components=20, method = 'picard', random_state=97)
ica.fit(raw_eeg, reject = None)

In [ ]:
ica.plot_components()
ica.plot_sources(raw_eeg)

In [ ]:
ica.exclude = [0, 1, 4]
cleaned_eeg = ica.apply(raw_eeg)
ica.plot_properties(raw_eeg, picks=ica.exclude)

In [ ]:
mne.viz.plot_raw(cleaned_eeg)

In [ ]:
### Eye-blink removal:

In [ ]:
eog_epochs = create_eog_epochs(cleaned_eeg) 
eog_inds, scores = ica.find_bads_eog(eog_epochs)

ica.exclude = eog_inds
exclude_blinks_eeg = ica.apply(cleaned_eeg)

In [ ]:
mne.viz.plot_raw(exclude_blinks_eeg)

In [ ]:
## Events:

In [ ]:
events = mne.find_events(raw_eeg, stim_channel='Status')

In [ ]:
print(events[:5])
events[:, 2] = events[:, 2] & 0xFF  # 0xFF = 255 = 11111111 in binary

print(events[:10])

In [ ]:
fig = mne.viz.plot_events(
    events, sfreq=raw_eeg.info["sfreq"], first_samp=raw_eeg.first_samp
)

## Events of interest (stimulus onset)

In [ ]:
import numpy as np

events_of_interest = [100, 125, 137, 145, 150, 155, 175, 163, 200]
event_ix = np.isin(events[:, 2], events_of_interest) 
stim_events = events[event_ix] # example event IDs

### Split file per block 

In [ ]:
# trigger 90 goes off at the beginning of each block
trigger_90_rows = events[events[:, 2] == 90]

In [ ]:
split_sample = trigger_90_rows[1, 0]  # sample index
split_time_sec = split_sample / raw_eeg.info['sfreq']

# Split raw EEG
raw_eeg_block1 = raw_eeg.copy().crop(tmin=0.0, tmax=split_time_sec)
raw_eeg_block2 = raw_eeg.copy().crop(tmin=split_time_sec, tmax=raw_eeg.times[-1])

# Split stim_events for each block
stim_events_block1 = stim_events[stim_events[:, 0] / raw_eeg.info['sfreq'] < split_time_sec]
stim_events_block2 = stim_events[stim_events[:, 0] / raw_eeg.info['sfreq'] >= split_time_sec]


## Epochs to look into ERPs - might be changed

In [ ]:
epochs_block1 = mne.Epochs(
    raw_eeg_block1,
    stim_events_block1,
    tmin=-0.2,
    tmax=0.8,
    preload=True
)


epochs_block2 = mne.Epochs(
    raw_eeg_block2,
    stim_events_block2,
    tmin=-0.2,
    tmax=0.8,
    preload=True
)

In [ ]:
mne.viz.plot_epochs(epochs_block1)
evoked_block1 = epochs_block1.average()
evoked_block1.plot()


mne.viz.plot_epochs(epochs_block2)
evoked_block2 = epochs_block2.average()
evoked_block2.plot()